In [4]:
#Please note that this is a Python 2 environment:
import wikipedia
import requests
import json
import wptools

In [5]:
def read_wiki_page(wikipage):
    url = "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles="+wikipage+"&format=json"
    ret=""
    req={}
    
    req["action"]="query"
    req["prop"]="revisions"
    req["rvprop"]="content"
    req["rvsection"]=0
    req["titles"]=wikipage
    req["format"]="json"
    
    
    r=requests.post(url, data=req )
    res = r.text
    res=json.loads(str(res))
    pages = res["query"]["pages"]
    for key, value in pages.items():
        ret=value["revisions"][0]["*"]
    
    return ret

In [6]:
def read_birthplace(page): #If the artist is a person, we need the birth place, if it's a band, their origin
    
    if 'birth_place' in page:
        pos_start=page.find('birth_place')
        pos_equal=page.find("=",pos_start)+1
        pos_end=page.find("\n",pos_equal)
        return str(page[pos_equal:pos_end])
    elif 'origin' in page:
        pos_start=page.find('origin')
        pos_equal=page.find("[[",pos_start)
        pos_end=page.find("\n",pos_equal)
        return str(page[pos_equal:pos_end])

def extract_genre(page):
    pos_start=page.find('genre')
    pos_equal=page.find("[[",pos_start)+2 #We extract the main genre for each artist
    pos_end=page.find("|",pos_equal)
    
    return str(page[pos_equal:pos_end])

In [7]:
page = wptools.page('Elton John')
page.get_query()

en.wikipedia.org (query) Elton John
en.wikipedia.org (query) Elton John (&plcontinue=5052197|0|Fatboy_Slim)
en.wikipedia.org (query) Elton John (&plcontinue=5052197|0|Nation...
en.wikipedia.org (query) Elton John (&plcontinue=5052197|0|Up_Whe...
en.wikipedia.org (imageinfo) File:Elton John 2011 Shankbone 2.JPG
Elton John (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> 00042308022 IPI, Reg Dwight, Reginald Kenneth...
  assessments: <dict(7)> England, Pop music, LGBT studies, Musical...
  description: English rock singer-songwriter, composer and pianist
  extext: <str(3491)> **Sir Elton Hercules John** (born **Reginald...
  extract: <str(3679)> <p class="mw-empty-elt"></p><p><b>Sir Elton...
  image: <list(8)> {u'size': 1158513, 'kind': 'query-pageimage', u...
  label: Elton John
  length: 147,138
  links: <list(1580)> (Everything I Do) I Do It for You, (I've Had...
  modified: <dict(1)> page
  pageid: 5052197
  random: Hayters Hill, New South Wales
  redirects: <list(32)>

In [8]:
page.get()

www.wikidata.org (wikidata) Q2808
www.wikidata.org (labels) P1375|P5121|P2267|P434|Q177220|Q1860|P4...
www.wikidata.org (labels) P742|P2850|P3192|Q170027|P5284|Q6166964...
www.wikidata.org (labels) P1207|P271|P3280|P358|P1828|Q855091|P63...
www.wikidata.org (labels) P21|P910|P950|P2397|P734|Q7135347|P1477...
+ query results in cache
en.wikipedia.org (parse) 5052197
en.wikipedia.org (restbase) /page/summary/Elton John
en.wikipedia.org (imageinfo) File:Elton John 2011 Shankbone 2.JPG
Elton John (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Reg Dwight, Sir Elton Hercules John, Reginald...
  assessments: <dict(7)> England, Pop music, LGBT studies, Musical...
  claims: <dict(114)> P646, P1375, P800, P2031, P2267, P434, P648,...
  description: English rock singer-songwriter, composer and pianist
  exhtml: <str(922)> <p><b>Sir Elton Hercules John</b> is an Engli...
  exrest: <str(832)> Sir Elton Hercules John is an English singer,...
  extext: <str(3491)> **Sir Elton Hercule

In [97]:
b =read_wiki_page('The Beatles')
read_birthplace(b)

'[[Liverpool]], England'

In [98]:
a =read_wiki_page('Metallica')
extract_genre(a)

'Heavy metal music'

In [90]:
read_birthplace(page)

' [[Jerez de la Frontera]], [[Spain]]'

In [101]:
extract_genre(page)

'Andalusia]]n folklore\n'

In [87]:
a = read_infobox(page)

In [148]:
read_birthplace(a)

' [[St. Louis|St. Louis, Missouri]], U.S.'

In [149]:
page=read_wiki_page("Adele")
read_birthplace(read_infobox(page))

' [[Tottenham]], London, England'

In [122]:
get_music_infobox(page)

u'{{Infobox musical artist <!-- See Wikipedia:WikiProject_Musicians -->\n| embed            = yes\n| background       = solo_singer\n| instrument       = Guitar, vocals\n| genre            = [[Rock and roll]]\n| years_active     = 1953\u20132016\n| label            = {{hlist|[[Chess Records|Chess]]|[[Mercury Records|Mercury]]|[[Atco Records|Atco]]|[[Dualtone Records|Dualtone]]}}\n| associated_acts  = {{hlist|[[Johnnie Johnson (musician)|Johnnie Johnson]]|[[T-Bone Walker]]|[[Muddy Waters]]}}\n}'

In [125]:
extract_genre(page)

'Rock and roll'

In [110]:
page2 = read_wiki_page("Elvis Presley")

In [114]:
get_music_infobox(page2)

u"{{Infobox musical artist|embed=yes\n | instrument   = {{flat list|\n* Vocals\n* guitar\n* piano\n}}<!--- If you think an instrument should be listed, a discussion to reach consensus is needed first per: https://en.wikipedia.org/wiki/Template:Infobox_musical_artist#instrument--->\n | background   = solo_singer\n | genre        = {{flat list|\n* [[Rock and roll]]\n* [[Pop music|pop]]\n* [[rockabilly]]\n* [[Country music|country]]\n* [[blues]]\n* [[Gospel music|gospel]]\n* [[rhythm and blues]]\n}}\n | label        = {{flatlist|\n* [[Sun Records|Sun]]\n* [[RCA Records|RCA (Victor)]]\n* [[His Master's Voice|HMV]]\n}}\n | years_active = 1953\u20131977\n | associated_acts = {{flatlist|\n* [[The Blue Moon Boys]]\n* [[The Jordanaires]]\n* [[The Imperials]]\n* [[Million Dollar Quartet]]\n}}\n}"

In [126]:
extract_genre(page2)

'Rock and roll'

In [168]:
get_music_infobox(page)

u"{{Infobox musical artist  <!-- See Wikipedia:WikiProject Musicians -->\n| name           = Lola Flores\n| image          = Estatua Lola Flores Jerez.jpg\n| caption        = Monument in her hometown, [[Jerez de la Frontera]]\n| background     = solo_singer\n| birth_name     = Mar\xeda Dolores Flores Ruiz\n| alias          = ''La Faraona''\n| birth_date     = 21 January 1923 \n|birth_place     = [[Jerez de la Frontera]], [[Spain]]\n| death_date     = {{Death date and age|1995|5|16|1923|1|21|df=y}}\n|"

In [75]:
page = read_wiki_page("Lola Flores")
get_music_infobox(page)

u"{{Infobox musical artist  <!-- See Wikipedia:WikiProject Musicians -->\n| name           = Lola Flores\n| image          = Estatua Lola Flores Jerez.jpg\n| caption        = Monument in her hometown, [[Jerez de la Frontera]]\n| background     = solo_singer\n| birth_name     = Mar\xeda Dolores Flores Ruiz\n| alias          = ''La Faraona''\n| birth_date     = 21 January 1923 \n|birth_place     = [[Jerez de la Frontera]], [[Spain]]\n| death_date     = {{Death date and age|1995|5|16|1923|1|21|df=y}}\n|"

## Extracting data for a list of artists and time it

In [23]:
import time
import pandas as pd
import numpy as np

In [99]:
artist = ["Lady Gaga", "Elton John", "Metallica", "Lola Flores"]
d = {'artist':artist}

In [100]:
data = pd.DataFrame(d)
data

,artist
0,Lady Gaga
1,Elton John
2,Metallica
3,Lola Flores


In [95]:
def retrieve_metadata(df=pd.DataFrame, serie=pd.Series):
    
    start = time.time()
    birth_place = []
    genre = []
    
    for i in serie:
        try:
            page = read_wiki_page(i)
            birth_place.append(read_birthplace(page))
            genre.append(extract_genre(page))
        except:
            birth_place.append(np.nan)
            genre.append(np.nan)
    
    df['birth_place'] = birth_place
    df['genre'] = genre

    end = time.time()
    print(end-start)
    
    return df

In [95]:
def retrieve_metadata(artist):
    
    start = time.time()
    birth_place = []
    genre = []
    
    for i in serie:
        try:
            page = read_wiki_page(i)
            birth_place.append(read_birthplace(page))
            genre.append(extract_genre(page))
        except:
            birth_place.append(np.nan)
            genre.append(np.nan)
    
    df['birth_place'] = birth_place
    df['genre'] = genre

    end = time.time()
    print(end-start)
    
    return df

In [101]:
retrieve_metadata(data, data['artist'])

1.88408899307


,artist,birth_place,genre
0,Lady Gaga,"[[New York City]], [[New York (state)|New Yor...",Pop music
1,Elton John,"[[Pinner]], [[Middlesex]], England",Rock music
2,Metallica,"[[Los Angeles]], [[California]], U.S.",Heavy metal music
3,Lola Flores,"[[Jerez de la Frontera]], [[Spain]]",Andalusia]]n folklore\n


In [158]:
start = time.time()
read_wiki_page('Metallica')
end = time.time()
print(end-start)

0.411010980606


In [11]:
import multiprocessing as mp

In [31]:
artists = pd.read_csv('Musicbrainz/Tables_used/artist.txt',sep='\t', header=None, engine='c', usecols=[0,3])
artists.columns = ['artist_id','artist_name']
artists.head()

,artist_id,artist_name
0,805192,WIK▲N
1,371203,"Moutso, Pete"
2,273232,Zachary
3,101060,"Silhouettes, The"
4,145773,"Leavitt, Aric"


In [67]:
#For the search in Wikipedia to run, we need to format the artists name:

def reverse(word):
    
    if ',' in word:
        b = word.split(',')
        rev = b[-1::-1]
        out = ' '.join(rev)
        return out
    
    else:
        return word

In [76]:
artists['name_formatted'] = artists['artist_name'].apply(lambda x: reverse(str(x)))

In [77]:
artists.head()

,artist_id,artist_name,name_formatted
0,805192,WIK▲N,WIK▲N
1,371203,"Moutso, Pete",Pete Moutso
2,273232,Zachary,Zachary
3,101060,"Silhouettes, The",The Silhouettes
4,145773,"Leavitt, Aric",Aric Leavitt


In [115]:
a = artists.head(100)
a.head()

,artist_id,artist_name,name_formatted
0,805192,WIK▲N,WIK▲N
1,371203,"Moutso, Pete",Pete Moutso
2,273232,Zachary,Zachary
3,101060,"Silhouettes, The",The Silhouettes
4,145773,"Leavitt, Aric",Aric Leavitt


In [96]:
retrieve_metadata(a, a['name_formatted'])

46.228908062


/home/ainhoa/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ainhoa/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,artist_id,artist_name,name_formatted,birth_place,genre
0,805192,WIK▲N,WIK▲N,NaN,NaN
1,371203,"Moutso, Pete",Pete Moutso,NaN,NaN
2,273232,Zachary,Zachary,"[[Zechariah (given name)|Zechariah]], [[Zakari...",{otheruses}}\n{{Infobox given name\n
3,101060,"Silhouettes, The",The Silhouettes,None,{About
4,145773,"Leavitt, Aric",Aric Leavitt,NaN,NaN
5,1076328,Fonograff,Fonograff,NaN,NaN
6,1172876,"Street, Al",Al Street,NaN,NaN
7,220155,Love .45,Love .45,NaN,NaN
8,618464,Sintellect,Sintellect,NaN,NaN
9,285714,Evie Tamala,Evie Tamala,"[[Tasikmalaya]], [[West Java]], [[Indonesia]]",Dangdut]]\n}}\n


In [117]:
#https://sebastianraschka.com/Articles/2014_multiprocessing.html
import multiprocessing as mp
# Define an output queue
output = mp.Queue()

def get_1_artist(artist, output):
    try:
        page = read_wiki_page(artist)
        birth_place= read_birthplace(page)
        genre = extract_genre(page)
    except:
        birth_place = np.nan
        genre = np.nan
    output.put([artist, birth_place, genre])

start = time.time()
processes = [mp.Process(target=get_1_artist, args=(name_formatted, output)) for name_formatted in a['name_formatted']]

for p in processes:
    p.start()

print("Processes started")

# Exit the completed processes
for p in processes:
    p.join()

results = [output.get() for p in processes]
end = time.time()
print(end-start)
print(results)

Processes started
2.14195990562
[[' Pete Moutso', nan, nan], ['WIK\xe2\x96\xb2N', nan, nan], [' Al Street', nan, nan], ['Fonograff', nan, nan], [' Aric Leavitt', nan, nan], ['Zachary', '[[Zechariah (given name)|Zechariah]], [[Zakariya]]', '{otheruses}}\n{{Infobox given name\n'], ['Sintellect', nan, nan], ['wecamewithbrokenteeth', nan, nan], [' The Blackbelt Band', nan, nan], [' Jean-Pierre Martin', nan, nan], [' The Silhouettes', None, '{About'], ['Giant Tomo', nan, nan], ['Evie Tamala', ' [[Tasikmalaya]], [[West Java]], [[Indonesia]]', 'Dangdut]]\n}}\n'], ['Diskobitch', nan, nan], ['Seanews', nan, nan], ['Infester', nan, nan], ['Decoding Jesus', nan, nan], ['CyberDog', None, 'REDIRECT [[Cyberdog]]\n\n{{Redirect category shell'], [' The Insignificant', nan, nan], ['Thirteen Over Eight', None, 'REDIRECT [[Innerpartysystem]'], ['Astolat', None, '[File:Elaine, by Alfred Tennyson 03.jpg'], ['Deejay One', nan, nan], ['Serge Bec', nan, nan], ['Sailing Conductors', nan, nan], ['Project/Object

In [107]:
a['name_formatted'][1]

' Pete Moutso'